In [30]:
import json
import urllib.parse
from pprint import pprint

import redis
from tqdm.auto import tqdm
from collections import defaultdict

db = redis.Redis(host="localhost", port=6379, db=1)

In [2]:
dataset = json.load(open("../data/logs_with_position.json"))

In [33]:
all_docs = {}
missing_docs = set()
for u in tqdm(dataset):
    for x in u["clicks"]:

        url = x["url"]
        if url in all_docs or url in missing_docs:
            continue
        if not db.exists(url):
            missing_docs.add(url)
            continue
        doc_text = db.get(url)

        all_docs[url] = doc_text

  0%|          | 0/127 [00:00<?, ?it/s]

In [6]:
with open("../data/missing_docs.txt", "w") as outf:
    for u in missing_docs:
        outf.write(f"{u}\n")

with open("../data/clicked_docs.tsv", "w") as outf:
    for (k, v) in tqdm(all_docs.items()):
        text = v.decode()
        clean_text = text.replace("[SEPREDIS]", " ").replace("\n", " ").replace("\t", " ")
        outf.write(f"{k}\t{clean_text}\n")

In [22]:
with open("../data/clicked_docs.tsv", "w") as outf:
    for (k, v) in tqdm(all_docs.items()):
        text = v.decode()
        clean_text = text.replace("[SEPREDIS]", " ").replace("\n", " ").replace("\t", " ")
        outf.write(f"{k}\t{clean_text}\n")

  0%|          | 0/1081 [00:00<?, ?it/s]

In [74]:
topic_texts = {urllib.parse.quote(x["topic_title"]): [] for x in dataset}

for line in open("../data/all_paragraphs.tsv"):
    title, text = line.strip().split("\t", maxsplit=1)
    topic = title.split("/")[0]
    if topic not in topic_texts.keys():
        continue
    topic_texts[topic].append(text)

In [80]:
with open("../data/wikipedia_texts.tsv", 'w') as outf:
    for k, v in topic_texts.items():
        text = " ".join(v)
        outf.write(f"{k}\t{text}")


In [75]:
[len(x) for x in topic_texts.values()]

[17, 10, 12, 3, 36, 10, 11]